# One Data Extract

TJ Kim
8/5/20

Extract from csv file ONE SIMULATOR data and group mobility.

The columns are as follows [time (s), node id (int), x coor (meter), y coor(meters), z coor (meters)]

The simulation takes place in a x=3925m and y = 2750m space.

The goal is to extract these columns per user and apply to stochastic movement.

In [1]:
import numpy as np

def get_one_sim_usr():
    """
    Get users and their average speeds in a list in onesim
    """
    # Import Data and eliminate z axis
    one_sim_data = np.genfromtxt('ONE_SIM.csv', delimiter=',')
    one_sim_data = one_sim_data[:,:4]

    # Data polishing
    meter2mile = 0.000621371
    # Convert Time from seconds --> 5 min intervals
    one_sim_data[:,0] *= (1/300)
    one_sim_data[:,0] = np.floor(one_sim_data[:,0])
    # Convert distance from meters --> Miles
    one_sim_data[:,2] *= meter2mile
    one_sim_data[:,3] *= meter2mile

    # Data Dividing and sampling
    ids = one_sim_data[:,1]
    unique_ids = np.unique(ids) 

    one_users = {}
    for idx in unique_ids:
        # Take id locations and take out relevant id data
        curr_locs = np.where(ids == idx)[0]
        start_loc, end_loc = curr_locs[0], curr_locs[-1]
        one_sim_data_u = one_sim_data[start_loc:end_loc,:]

        # Sampling (time seems to go from zero to 36)
        times = np.arange(0,37)
        sampled_data = np.zeros((times.shape[0],5))

        for t in times:
            # print("idx,t", idx, t)
            curr_time_loc = np.where(one_sim_data_u == t)[0]
            if curr_time_loc.size != 0:
                sampled_data[t,0:4] = one_sim_data_u[curr_time_loc[0],:]
            else:
                sampled_data[t,0:4] = sampled_data[t-1,0:4]
                sampled_data[t,0] = t

            # Find mean displacement from prev step
            if t > 0:
                curr_loc = sampled_data[t,2:4]
                prev_loc = sampled_data[t-1,2:4]
                diff = curr_loc - prev_loc
                summed = np.sqrt(diff[0]**2 + diff[1]**2)
                sampled_data[t,4] = summed

        one_users[idx] = sampled_data

    mean_spd_list = []
    for key in one_users:
        usr = one_users[key]
        mean_spd_list += [np.mean(usr[:,4])]
        
    return one_users, mean_spd_list

In [2]:
one,two = get_one_sim_usr()

In [3]:
one

{30.0: array([[0.00000000e+00, 3.00000000e+01, 6.04494564e-01, 1.30612184e+00,
         0.00000000e+00],
        [1.00000000e+00, 3.00000000e+01, 6.79779874e-01, 1.17420478e+00,
         1.51888082e-01],
        [2.00000000e+00, 3.00000000e+01, 4.06855090e-01, 1.25342958e+00,
         2.84190970e-01],
        [3.00000000e+00, 3.00000000e+01, 3.33346900e-01, 1.38143201e+00,
         1.47607842e-01],
        [4.00000000e+00, 3.00000000e+01, 4.70098230e-01, 1.32886402e+00,
         1.46507063e-01],
        [5.00000000e+00, 3.00000000e+01, 6.79779874e-01, 1.17420478e+00,
         2.60549176e-01],
        [6.00000000e+00, 3.00000000e+01, 5.06392510e-01, 1.36782398e+00,
         2.59906856e-01],
        [7.00000000e+00, 3.00000000e+01, 4.59640556e-01, 1.31419967e+00,
         7.11429028e-02],
        [8.00000000e+00, 3.00000000e+01, 4.57664596e-01, 1.15587433e+00,
         1.58337661e-01],
        [9.00000000e+00, 3.00000000e+01, 4.57664596e-01, 1.15587433e+00,
         0.00000000e+00],
    